# Data processing

### Executive summary
#### This script performs pre processing of the original datasets and performs statistic analysis on the pre-processed dataset

In [ ]:
import pandas as pd
import numpy as np
import quandl
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import math
import datetime as dt

# 1 Original Dataset import and preprocessing
## To be executed only in case of modifications to the original dataset

## 1.1 Import of data

In [ ]:
df = pd.read_excel(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\DATA\sample_trades_intraday_DE.xlsx')

In [ ]:
df = pd.read_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\DATA\sample_trades_intraday_DE_complete.csv',parse_dates=['timestamp','datetime'])
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['day_week'] = df['timestamp'].dt.day_name()
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['second'] = df['timestamp'].dt.second
df = df.set_index('timestamp')
df.sort_values(by=['timestamp'])
df['timestamp'] = df.index


In [ ]:
df = df.rename(columns = {'Unnamed: 0':'numeric_index'})

In [ ]:
df['numeric_index'] = df.index

In [ ]:
df_original = df

In [ ]:
df = df.sample(frac=1)

In [ ]:
split_size = 1
df_split = np.array_split(df, split_size)

In [ ]:
print(df.info())

In [ ]:
sct.describe(df.price)

In [ ]:
df = df_split

## 1.2 Generation of a working dataframe

We want to keep **max one datapoint per second**. Therefore, we take the average of the volume and of the price of datapoints that have the same timestamp.

In [ ]:
# We initialize the columns of our new dataframe
volume = []
price = []
timestamp = []
datetime = []

for i in range(0,1000):
    # We select only the unique timestamps
    unique_timestamps = df[i].timestamp.unique()

    #i = 0
    # we cycle on the unique timestamps to create the new columns
    for t in unique_timestamps:
        #j=j+1
        # we select all the rows that correspond to the same timestamp
        mask = (df[i].timestamp == t)

        # We average the volumes
        volume.append(np.mean(df[i].volume[mask]))

        # We average the prices
        price.append(np.mean(df[i].price[mask]))
        
        # we append the timestamps
        timestamp.append(t)
        
        # We append the datetime
        if (len(df[i].numeric_index[mask])>1):
            
            location = np.array(df[i].numeric_index[mask])[0]
        else:
            location = float(df[i].numeric_index[mask])
            
        #starting_index = df[i].datetime.index[0]
        #location_index = np.where(mask == False)[0][0]
        datetime.append(df[i].datetime[location])

In [ ]:
# We initialize the columns of our new dataframe
volume = []
price = []
timestamp = []
datetime = []


# We select only the unique timestamps
unique_timestamps = df.timestamp.unique()

#i = 0
# we cycle on the unique timestamps to create the new columns
for t in unique_timestamps:
#j=j+1
# we select all the rows that correspond to the same timestamp
    mask = (df.timestamp == t)

    # We average the volumes
    volume.append(np.mean(df.volume[mask]))

    # We average the prices
    price.append(np.mean(df.price[mask]))

    # we append the timestamps
    timestamp.append(t)

    # We append the datetime
    if (len(df.numeric_index[mask])>1):

        location = np.array(df.numeric_index[mask])[0]
    else:
        location = float(df.numeric_index[mask])

    #starting_index = df[i].datetime.index[0]
    #location_index = np.where(mask == False)[0][0]
    datetime.append(df.datetime[location])

In [ ]:
# We now create our new dataframe containing only unique values per every second
df_new = {'datetime': datetime,
          'volume': volume,
          'price': price,
          'timestamp': timestamp}
# We want our dataframe to be sorted according to the timestamp (from the oldest to the newest datapoint)
df_new = pd.DataFrame(df_new, index=timestamp)
df_new = df_new.sort_values(by='timestamp')

# We also want to identify  the different days and separate them from each other
dates = df_new['timestamp'].dt.date.unique()

In [ ]:
print('Rows original dataframe: ',len(df_original.timestamp))
print('Rows new dataframe: ',len(df_new.timestamp))
print('Unique timestamps original dataframe: ',len(df_original.timestamp.unique()))
print('Unique timestamps new dataframe: ',len(df_new.timestamp.unique()))

In [ ]:
print(df_new.info())

In [ ]:
df_new.to_csv('EPEX_small_preprocessed.csv')

In [ ]:
# We select only the unique timestamps
unique_timestamps = df.timestamp.unique()

# We initialize the columns of our new dataframe
volume = []
price = []
timestamp = []
datetime = []

# we cycle on the unique timestamps to create the new columns
for t in unique_timestamps:
    
    # we select all the rows that correspond to the same timestamp
    mask = (df.timestamp == t)
    
    # We average the volumes
    volume.append(np.mean(df.volume[mask]))
    
    # We average the prices
    price.append(np.mean(df.price[mask]))
    
    # we append the timestamps
    timestamp.append(t)
    starting_index = df.datetime.index[0]
    location_index = np.where(mask == False)[0][0]
    datetime.append(starting_index + location_index)

## 2 Import of pre-processed dataframes

## 2.1 Import of the two datasets

### 2.1.1 Small EPEX dataset (5 days)

In [ ]:
df_new_small = pd.read_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\DATA\EPEX_small_preprocessed.csv',parse_dates=['timestamp','datetime'])
df_new_small = df_new_small.set_index('timestamp')
del df_new_small['Unnamed: 0']
df_new_small['timestamp'] = df_new_small.index

### 2.1.2 Including information from the dates (5 days)

In [ ]:
# We extract useful details from the timestamp and we add them to the dataframe
df_new_small['month'] = df_new_small['timestamp'].dt.month
df_new_small['day'] = df_new_small['timestamp'].dt.day
df_new_small['day_week'] = df_new_small['timestamp'].dt.day_name()
df_new_small['hour'] = df_new_small['timestamp'].dt.hour
df_new_small['minute'] = df_new_small['timestamp'].dt.minute
df_new_small['second'] = df_new_small['timestamp'].dt.second

### 2.2.1 Big EPEX dataset (2 months)

In [ ]:
df_new_big = pd.read_csv(r'C:\Users\luigi\OneDrive\2 University\4 ULB\0 THESIS\DATA\EPEX_big_preprocessed.csv',parse_dates=['timestamp','datetime'])
df_new_big = df_new_big.set_index('timestamp')
del df_new_big['Unnamed: 0']
df_new_big['timestamp'] = df_new_big.index

### 2.2.2 Including information from the dates (2 months)

In [ ]:
# We extract useful details from the timestamp and we add them to the dataframe
df_new_big['month'] = df_new_big['timestamp'].dt.month
df_new_big['day'] = df_new_big['timestamp'].dt.day
df_new_big['day_week'] = df_new_big['timestamp'].dt.day_name()
df_new_big['hour'] = df_new_big['timestamp'].dt.hour
df_new_big['minute'] = df_new_big['timestamp'].dt.minute
df_new_big['second'] = df_new_big['timestamp'].dt.second

## 2.2 Choice of the dataset for the resampling

In [ ]:
df_new = df_new_big

## 2.3 Derived averaged datasets

### 2.3.1 60s average (one minute)

In [ ]:
# We identify the number of missing rows
df_new_minute = df_new.resample('min').mean()
NAN_60s = pd.isna(df_new_minute.price)
NAN_60s_tot = np.sum(NAN_60s)


df_new_minute = df_new.resample('min').mean().bfill()
df_new_minute['timestamp'] = df_new.timestamp.resample('min').bfill()
df_new_minute['datetime'] = df_new.datetime.resample('min').bfill()

print('Total number of values filled: ', NAN_60s_tot ,' Corresponding to ', NAN_60s_tot/len(df_new_minute.price)*100, '% of the total') 

In [ ]:
plt.figure(figsize = (20,10))

# Values to plot
plt.plot(df_new_minute.price, label = 'Prices 1-minute average')
plt.plot(df_new_minute.price[NAN_60s],color = 'r',marker='o',markersize=1,linewidth=0, label = 'samples filled')


plt.title('1-minute average',fontsize=30,backgroundcolor= 'white')
plt.xticks(fontsize=20,backgroundcolor= 'white')
plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('intraday price [€/MWh]',fontsize=25,backgroundcolor= 'white')
plt.xlabel('time',fontsize=25,backgroundcolor= 'white')
plt.grid()    
plt.legend(fontsize=20)
plt.show()

### 2.3.2 30s average

In [ ]:
df_new_30s = df_new.resample('30S').mean()
NAN_30s = pd.isna(df_new_30s.price)
NAN_30s_tot = np.sum(NAN_30s)

df_new_30s = df_new.resample('30S').mean().bfill()
df_new_30s['timestamp'] = df_new.timestamp.resample('30S').bfill()
df_new_30s['datetime'] = df_new.datetime.resample('30S').bfill()
#df_new_30s['second'] = np.floor(df_new_30s.second/30)*30

print('Total number of values filled: ', NAN_30s_tot ,' Corresponding to ', NAN_30s_tot/len(df_new_30s.price)*100, '% of the total') 

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(df_new_30s.price, label = 'Prices 30s average')
plt.plot(df_new_30s.price[NAN_30s],color = 'r',marker='o',markersize=1, linewidth=0, label = 'samples filled')

plt.title('30s average',fontsize=30,backgroundcolor= 'white')
plt.xticks(fontsize=20,backgroundcolor= 'white')
plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('intraday price [€/MWh]',fontsize=25,backgroundcolor= 'white')
plt.xlabel('time',fontsize=25,backgroundcolor= 'white')
plt.grid()    
plt.legend(fontsize=20)
plt.show()

### 2.3.3 15s average

In [ ]:
df_new_15s = df_new.resample('15S').mean()
NAN_15s = pd.isna(df_new_15s.price)
NAN_15s_tot = np.sum(NAN_15s)

df_new_15s = df_new.resample('15S').mean().bfill()
df_new_15s['timestamp'] = df_new.timestamp.resample('15S').bfill()
df_new_15s['datetime'] = df_new.datetime.resample('15S').bfill()
#df_new_30s['second'] = np.floor(df_new_30s.second/30)*30

print('Total number of values filled: ', NAN_15s_tot ,' Corresponding to ', NAN_15s_tot/len(df_new_15s.price)*100, '% of the total') 

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(df_new_15s.price,label = 'Prices 15s average')
plt.plot(df_new_15s.price[NAN_15s],color = 'r',marker='o',markersize=1, linewidth=0,label = 'samples filled')

plt.title('15s average',fontsize=30,backgroundcolor= 'white')
plt.xticks(fontsize=20,backgroundcolor= 'white')
plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('intraday price [€/MWh]',fontsize=25,backgroundcolor= 'white')
plt.xlabel('time',fontsize=25,backgroundcolor= 'white')
plt.grid()    
plt.legend(fontsize=20)
plt.show()

### 2.3.4 1s expansion

In [ ]:
df_new_1s = df_new.resample('S').mean()
NAN_1s = pd.isna(df_new_1s.price)
NAN_1s_tot = np.sum(NAN_1s)

df_new_1s = df_new.resample('S').mean().bfill()
df_new_1s['timestamp'] = df_new.timestamp.resample('1S').bfill()
df_new_1s['datetime'] = df_new.datetime.resample('1S').bfill()
print('Total number of values filled: ', NAN_1s_tot ,' Corresponding to ', NAN_1s_tot/len(df_new_1s.price)*100, '% of the total') 

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(df_new_1s.price, label = 'Prices 1s average')
plt.plot(df_new_1s.price[NAN_1s],color = 'r',marker='o',markersize=1, linewidth=0, label = 'samples filled')

plt.title('1s Expansion',fontsize=30,backgroundcolor= 'white')
plt.xticks(fontsize=20,backgroundcolor= 'white')
plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('intraday price [€]',fontsize=25,backgroundcolor= 'white')
plt.xlabel('time',fontsize=25,backgroundcolor= 'white')
plt.grid()    
plt.legend(fontsize=20)
plt.show()

### 2.3.5 5 minutes averages

In [ ]:
df_new_5min = df_new.resample('5min').mean()
NAN_5min = pd.isna(df_new_5min.price)
NAN_5min_tot = np.sum(NAN_5min)

df_new_5min = df_new.resample('5min').mean().bfill()
df_new_5min['timestamp'] = df_new.timestamp.resample('5min').bfill()
df_new_5min['datetime'] = df_new.datetime.resample('5min').bfill()

# 2.3.6 10 minutes average

In [ ]:
df_new_10min = df_new.resample('10min').mean()
NAN_10min = pd.isna(df_new_10min.price)
NAN_10min_tot = np.sum(NAN_10min)

df_new_10min = df_new.resample('10min').mean().bfill()
df_new_10min['timestamp'] = df_new.timestamp.resample('10min').bfill()
df_new_10min['datetime'] = df_new.datetime.resample('10min').bfill()

In [ ]:
df_new_10min

### 2.3.7 15 minutes average

In [ ]:
df_new_15min = df_new.resample('15min').mean()
NAN_15min = pd.isna(df_new_15min.price)
NAN_15min_tot = np.sum(NAN_15min)

df_new_15min = df_new.resample('15min').mean().bfill()
df_new_15min['timestamp'] = df_new.timestamp.resample('15min').bfill()
df_new_15min['datetime'] = df_new.datetime.resample('15min').bfill()

### 2.3.8 30 minutes average

In [ ]:
df_new_30min = df_new.resample('30min').mean()
NAN_30min = pd.isna(df_new_30min.price)
NAN_30min_tot = np.sum(NAN_30min)

df_new_30min = df_new.resample('30min').mean().bfill()
df_new_30min['timestamp'] = df_new.timestamp.resample('30min').bfill()
df_new_30min['datetime'] = df_new.datetime.resample('30min').bfill()

#### Save the pre-processed datasets

In [ ]:
df_new_big.to_csv('DF_big_processed.csv')
df_new_30s.to_csv('df_30s_processed.csv')
df_new_15s.to_csv('df_15s_processed.csv')
df_new_minute.to_csv('df_minute_processed.csv')

In [ ]:
df_new_5min.to_csv('df_5min_processed.csv')
df_new_10min.to_csv('df_10min_processed.csv')
df_new_15min.to_csv('df_15min_processed.csv')
df_new_30min.to_csv('df_30min_processed.csv')

### 2.3.2 10-minute average (to be fixed)

In [ ]:
# The list of all the timestamps
stamps = df_new.timestamp

# We create an empty list for the averaged volume prices
weighted_volume_prices_averaged = []
for time in stamps: 
    central_time = time


In [ ]:
# The list of all the timestamps
stamps = df_new.timestamp

# We create an empty list for the averaged volume prices
weighted_volume_prices_averaged = []

# We iterate on every datapoint calculating the 10-minutes averages of the price weighted on the volume
for time in stamps: 
    central_time = time
    
    # The datapoint 5 minutes ahead the current point
    five_min_up = pd.Timestamp(central_time) + dt.timedelta(minutes = 5)
    
    # The datapoint 5 minutes behing the current data point
    five_min_down = pd.Timestamp(central_time) - dt.timedelta(minutes = 5)
    
    # The volume weighted average of the electricity price (10-minute based)
    weighted_volume_prices_averaged.append(np.average(df_new.price[five_min_down:five_min_up], 
                                                      weights=df_new.volume[five_min_down:five_min_up]))

# we add the volume weighted average to the dataframe
df_new['weighted_volume_prices_averaged'] = weighted_volume_prices_averaged

See resample reference: https://towardsdatascience.com/resample-function-of-pandas-79b17ec82a78

## 2.4 Plots

### 2.4.3 Overlapped daily graphs

In [ ]:
dates = df_new['timestamp'].dt.date.unique()

In [ ]:
# We can plot the days overlapped to each other
plt.subplots(figsize=(20, 10))
for date in dates:

    mask = df_new['timestamp'].dt.date == date
    x = df_new.timestamp.dt.time[mask]
    plt.plot(x,df_new.price[mask], label = date)

plt.title('Full dataset of intraday prices - overlapped',fontsize=30,backgroundcolor= 'white')
plt.xticks(fontsize=20,backgroundcolor= 'white')
plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('intraday price [€]',fontsize=25,backgroundcolor= 'white')
plt.xlabel('time',fontsize=25,backgroundcolor= 'white')
plt.grid()    
#plt.legend(fontsize=15)
plt.show()

## 2.4.4 Days in sequence

In [ ]:
plt.subplots(figsize=(20, 10))
for date in dates:

    mask = df_new['timestamp'].dt.date == date
    plt.plot(df_new.price[mask], label = date)
    
#plt.plot(df_new.weighted_volume_prices_averaged, label = '10 minutes prices average weighted on the volume')
#plt.plot(df_new_minute.price, 'r', label = 'minute average',)
plt.title('Full dataset of intraday prices',fontsize=30,backgroundcolor= 'white')
plt.xticks(fontsize=20,backgroundcolor= 'white')
plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('intraday price [€/MWh]',fontsize=25,backgroundcolor= 'white')
plt.xlabel('time',fontsize=25,backgroundcolor= 'white')
#plt.ylim(-200,250)
plt.grid()    
#plt.legend(fontsize=15)

plt.show()

In [ ]:
dates_big = df_new_big['timestamp'].dt.date.unique()
plt.subplots(figsize=(20, 10))
for date in dates_big:

    mask = df_new_big['timestamp'].dt.date == date
    plt.plot(df_new_big.price[mask], label = date)
    
#plt.plot(df_new.weighted_volume_prices_averaged, label = '10 minutes prices average weighted on the volume')
#plt.plot(df_new_minute.price, 'r', label = 'minute average',)
plt.title('Full dataset of intraday prices',fontsize=30,backgroundcolor= 'white')
plt.xticks(fontsize=20,backgroundcolor= 'white',rotation=45, ha="right")
plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('intraday price [€/MWh]',fontsize=25,backgroundcolor= 'white')
plt.xlabel('time',fontsize=25,backgroundcolor= 'white')
plt.ylim(-200,250)
plt.grid()    
#plt.legend(fontsize=15)

plt.show()

### 2.4.5 Estimation of the trend

In [ ]:
from sklearn.linear_model import LinearRegression

# Initialization of the model, a linear regression with intercept
model = LinearRegression(fit_intercept = True, normalize = True)

#### Generations of the arrays for the linear regression

In [ ]:
x1 = np.array([range(0,len(df_new.price))]).T

# Prices are converted into int for memory allocation reasons
y = df_new.price.astype(int)

# Fitting of the model and estimation of the predictions
model.fit(x1, y)
prediction = model.predict(x1)

#### plot of the trend

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(df.price, label = 'Intraday prices')
plt.plot(df.index, prediction, color='r', label = 'Linear regression trend', linewidth=5)

slope = round(model.coef_[0],7)
plt.title('Trend of the dataset, estimated with linear regression. Slope = %s ' % slope,fontsize=30,backgroundcolor= 'white')
plt.xticks(fontsize=20,backgroundcolor= 'white',rotation=45, ha="right")
plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('intraday price [€/MWh]',fontsize=25,backgroundcolor= 'white')
plt.xlabel('time',fontsize=25,backgroundcolor= 'white')
plt.grid()    
plt.legend(fontsize=20)
plt.show()

### 2.4.5 Display of the high volatility of the dataset (to be fixed)

In [ ]:
# We choose a certain timeframe to show in the plot for each of the datasets that we want to plot

mask = (df_new_small.month == 6) &(df_new_small.day == 2) & (df_new_small.hour == 15)
mask_30s = (df_new_small.month == 6) & (df_new_30s.day == 2) & (df_new_30s.hour == 15)
mask_15s = (df_new_small.month == 6) & (df_new_15s.day == 2) & (df_new_15s.hour == 15)
mask_minute = (df_new_small.month == 6) & (df_new_minute.day == 2) & (df_new_minute.hour == 15)


plt.subplots(figsize=(20, 10))
# We plot the original dataset and the averages
plt.plot(df_new_small.price[mask],'y', label='Original dataset')
plt.plot(df_new_30s.price[mask_30s],'r',label='30s average')
plt.plot(df_new_15s.price[mask_15s],'b',label='15s average')
plt.plot(df_new_minute.price[mask_minute],'g',label='Minute average')

# Items of the plot
plt.title('Variance of the dataset',fontsize=30,backgroundcolor= 'white')
plt.xticks(fontsize=20,backgroundcolor= 'white')
plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('intraday price [€/MWh]',fontsize=25,backgroundcolor= 'white')
plt.xlabel('time',fontsize=25,backgroundcolor= 'white')
plt.grid()    
plt.legend(fontsize=25)
plt.show()


#### Only 10 minutes time interval

In [ ]:
mask_short = (df_new_big.month == 6) &(df_new_big.day == 2) & (df_new_big.hour == 15) & (df_new_big.minute < 10)
mask_30s_short = (df_new_big.month == 6) & (df_new_30s.day == 2) & (df_new_30s.hour == 15) & (df_new_big.minute < 10)
mask_15s_short = (df_new_big.month == 6) & (df_new_15s.day == 2) & (df_new_15s.hour == 15) & (df_new_big.minute < 10)
mask_minute_short = (df_new_big.month == 6) & (df_new_minute.day == 2) & (df_new_minute.hour == 15) & (df_new_big.minute < 10)

plt.subplots(figsize=(20, 10))
# We plot the original dataset and the averages
plt.plot(df_new_big.price[mask_short],'y', label='Original dataset')
plt.plot(df_new_30s.price[mask_30s_short],'r',label='30s average')
plt.plot(df_new_15s.price[mask_15s_short],'b',label='15s average')
plt.plot(df_new_minute.price[mask_minute_short],'g',label='Minute average')

# Items of the plot
plt.title('Variance of the dataset',fontsize=30,backgroundcolor= 'white')
plt.xticks(fontsize=20,backgroundcolor= 'white')
plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('intraday price [€/MWh]',fontsize=25,backgroundcolor= 'white')
plt.xlabel('time',fontsize=25,backgroundcolor= 'white')
plt.grid()    
plt.legend(fontsize=25)
plt.show()

In [ ]:
### 2.4.6 experiments trying to fill the dataset with NA or with forward fill

In [ ]:
df_new2 = df_new1.interpolate(method='ffill', limit_direction='forward')
plt.subplots(figsize=(20, 10))

plt.plot(df_new.price[mask],'b', label = 'original dataset')
#plt.plot(df_new_minute.price[mask1],'r', label = 'minute average')
#plt.plot(df_new1.price[mask2],'g', label = 'dataset with every second, filled NA')
plt.plot(df_new2.price[mask2], 'y', label = 'dataset with every second, filled ffa')

In [ ]:
plt.subplots(figsize=(20, 10))

plt.plot(df_new.price[mask_short],'b', label = 'original dataset')
plt.plot(df_new_minute.price[mask1_short],'r', label = 'minute average')
#plt.plot(df_new1.price[mask2],'g', label = 'dataset with every second, filled NA')
plt.plot(df_new2.price[mask2_short], 'y', label = 'dataset with every second, filled ffa')

# 3 Statistics about the dataset

In [ ]:
df = df_new_big

In [ ]:
import scipy.stats as sct

In [ ]:
basic_stats = sct.describe(df.price)
print(basic_stats)


## 3.1 Histogram of the data

In [ ]:
N = len(df.price)

Number of bins: rule of **Freedman–Diaconis**

In [ ]:
h = sct.iqr(df_new_big.price) * 2 * N**(-1/3)
n_bins = int(np.floor((np.max(df.price)-np.min(df.price))/h))
n_bins

In [ ]:
plt.subplots(figsize=(20, 10))
# We plot the original dataset and the averages
plt.hist(df.price, bins = n_bins, label='Intraday prices')
plt.axvline(x=basic_stats.mean,c='r', label='mean')
#plt.ylim(0,10)

# Items of the plot
plt.xlim(-400+basic_stats.mean,400++basic_stats.mean)
plt.title('Histogram of the dataset',fontsize=30,backgroundcolor= 'white')
#plt.xticks(fontsize=20,backgroundcolor= 'white')
#plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('Frequency',fontsize=25,backgroundcolor= 'white')
plt.xlabel('Intraday price [€/MWh]',fontsize=25,backgroundcolor= 'white')
#plt.grid()    
plt.legend(fontsize=25)
plt.show()

### 3.2.1 Histogram of the Zoomed dataset

In [ ]:
plt.subplots(figsize=(20, 10))
# We plot the original dataset and the averages
plt.hist(df.price, bins = n_bins, label='Intraday prices')
plt.axvline(x=basic_stats.mean,c='r', label='mean')
plt.ylim(0,10)
# Items of the plot
plt.title('Histogram of the dataset - extreme values and outliers',fontsize=30,backgroundcolor= 'white')
#plt.xticks(fontsize=20,backgroundcolor= 'white')
#plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('Frequency',fontsize=25,backgroundcolor= 'white')
plt.xlabel('Intraday price [€]',fontsize=25,backgroundcolor= 'white')
#plt.grid()    
plt.legend(fontsize=25)
plt.show()

## 3.2 Analysis of the days of the week

In [ ]:
df['dayname'] = df.timestamp.dt.day_name()

### 3.2.1 Calculation of the stats for each day of the week

In [ ]:
dayweek_stats = []
for dayweek in (df.dayname.unique()):
    print(dayweek)
    dayweek_mask = (df.dayname == dayweek)
    dayweek_stats.append(sct.describe(df.price[dayweek_mask]))
    print(sct.describe(df.price[dayweek_mask]))

### 3.2.2 plot of the histograms with the mean for each day of the week

In [ ]:
plt.subplots(figsize=(20, 10))

# Monday
dayweek_mask = (df.dayname == 'Monday')
df_week = df[dayweek_mask]

N = len(df_week.price)
h = sct.iqr(df_week.price) * 2 * N**(-1/3)
n_bins = int(np.floor((np.max(df_week.price)-np.min(df_week.price))/h))
plt.hist(df_week.price, bins = n_bins,alpha=0.5, color='r',label = 'Monday')
plt.axvline(x=np.mean(df_week.price),c='r')


# Tuesday
dayweek_mask = (df.dayname == 'Tuesday')
df_week = df[dayweek_mask]

N = len(df_week.price)
h = sct.iqr(df_week.price) * 2 * N**(-1/3)
n_bins = int(np.floor((np.max(df_week.price)-np.min(df_week.price))/h))
plt.hist(df_week.price, bins = n_bins,alpha=0.5, color='darkorange',label = 'Tuesday')
plt.axvline(x=np.mean(df_week.price),c='darkorange')


# Wednesday
dayweek_mask = (df.dayname == 'Wednesday')
df_week = df[dayweek_mask]

N = len(df_week.price)
h = sct.iqr(df_week.price) * 2 * N**(-1/3)
n_bins = int(np.floor((np.max(df_week.price)-np.min(df_week.price))/h))
plt.hist(df_week.price, bins = n_bins,alpha=0.5, color='gold',label = 'Wednesday')
plt.axvline(x=np.mean(df_week.price),c='gold')

# Thursday
dayweek_mask = (df.dayname == 'Thursday')
df_week = df[dayweek_mask]

N = len(df_week.price)
h = sct.iqr(df_week.price) * 2 * N**(-1/3)
n_bins = int(np.floor((np.max(df_week.price)-np.min(df_week.price))/h))
plt.hist(df_week.price, bins = n_bins,alpha=0.5, color='lawngreen',label = 'Thursday')
plt.axvline(x=np.mean(df_week.price),c='lawngreen')

# Friday
dayweek_mask = (df.dayname == 'Friday')
df_week = df[dayweek_mask]

N = len(df_week.price)
h = sct.iqr(df_week.price) * 2 * N**(-1/3)
n_bins = int(np.floor((np.max(df_week.price)-np.min(df_week.price))/h))
plt.hist(df_week.price, bins = n_bins,alpha=0.5, color='cyan', label = 'Friday')
plt.axvline(x=np.mean(df_week.price),c='cyan')

# Saturday
dayweek_mask = (df.dayname == 'Saturday')
df_week = df[dayweek_mask]

N = len(df_week.price)
h = sct.iqr(df_week.price) * 2 * N**(-1/3)
n_bins = int(np.floor((np.max(df_week.price)-np.min(df_week.price))/h))
plt.hist(df_week.price, bins = n_bins,alpha=0.5, color='midnightblue',label = 'Saturday')
plt.axvline(x=np.mean(df_week.price),c='midnightblue')

# Sunday
dayweek_mask = (df.dayname == 'Sunday')
df_week = df[dayweek_mask]

N = len(df_week.price)
h = sct.iqr(df_week.price) * 2 * N**(-1/3)
n_bins = int(np.floor((np.max(df_week.price)-np.min(df_week.price))/h))
plt.hist(df_week.price, bins = n_bins,alpha=0.2, color='magenta', label = 'Sunday')
plt.axvline(x=np.mean(df_week.price),c='magenta')

# Items of the plot
plt.xlim(-200,200)

plt.title('Histogram of the dataset - days of the week',fontsize=30,backgroundcolor= 'white')
#plt.xticks(fontsize=20,backgroundcolor= 'white')
#plt.yticks(fontsize=20,backgroundcolor= 'white')
plt.ylabel('Frequency',fontsize=25,backgroundcolor= 'white')
plt.xlabel('Intraday price [€/MWh]',fontsize=25,backgroundcolor= 'white')
plt.legend(fontsize=25)
plt.show()

### Boxplots

In [ ]:
months = df.month.unique()
days = df.day.unique()

daily_stats = []
day_stats = []
month_stats = []
minprice = []
maxprice = []
for month in months:
    
    for day in days:
        mask_day = (df.month == month) & (df.day == day)
        if len(df.price[mask_day]) == 0:
            continue
        stats = sct.describe(df.price[mask_day])
        daily_stats.append(stats)
        day_stats.append(day)
        month_stats.append(month)
        minprice.append(np.min(df.price[mask_day]))
        maxprice.append(np.max(df.price[mask_day]))

In [ ]:
min_max_differences = np.array(maxprice)-np.array(minprice)

N = len(min_max_differences)
h = sct.iqr(min_max_differences) * 2 * N**(-1/3)
n_bins = int(np.floor((np.max(min_max_differences)-np.min(min_max_differences))/h))

plt.hist(min_max_differences, bins = 30, alpha=0.5, color='magenta')